In [1]:
!pip3 install requests
!pip3 install python-dotenv
!pip3 install tqdm



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
API_KEY = "TelLaRZgHhp5c7U8QE3lDAB83A1lxqOS"


In [4]:
import requests
!pip install pandas
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm 
import time

  Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached numpy-2.1.3-cp313-cp313-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl (11.3 MB)
Using cached numpy-2.1.3-cp313-cp313-macosx_14_0_arm64.whl (5.1 MB)
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:

startDate = datetime(2017, 1, 1)
endDate = datetime(2017,12,31)

def getArticlesForDate(date):
    endpoint = "https://api.nytimes.com/svc/search/v2/articlesearch.json"
    params = {
          "q": "U.S. AND (economy OR trade OR Federal Reserve OR policy OR Canada)",
          "fq": 'section_name:("U.S." OR "Business" OR "Economy")',
          "begin_date": date.strftime("%Y%m%d"),
          "end_date": date.strftime("%Y%m%d"),
          "sort": "relevance", 
          "api-key": API_KEY
        }

    response = requests.get(endpoint, params=params)

    if response.status_code == 200:
        return response.json()
    
    else:
        print(f"Error: {response.status_code} - {response.text}")

totalDays = (endDate - startDate).days + 1
articlesData = []

with tqdm(total=totalDays, desc="Fetching Articles") as pbar:
    current_date = startDate
    while current_date <= endDate:
        data = getArticlesForDate(current_date)
        if data:
            # Extract article details
            docs = data.get('response', {}).get('docs', [])
            for doc in docs:
                article_info = {
                    "title": doc.get("headline", {}).get("main", ""),
                    "snippet": doc.get("snippet", ""),
                    "url": doc.get("web_url", ""),
                    "published_date": doc.get("pub_date", ""),
                    "section": doc.get("section_name", ""),
                    "subsection": doc.get("subsection_name", ""),
                }
                articlesData.append(article_info)
        # Move to the next day and update the progress bar
        current_date += timedelta(days=1)
        pbar.update(1)
        time.sleep(7) 

articledf = pd.DataFrame(articlesData)
print(articledf)


Fetching Articles:   2%|▏         | 6/366 [00:38<38:20,  6.39s/it]

Error: 429 - {"fault":{"faultstring":"Rate limit quota violation. Quota limit  exceeded. Identifier : 2f21ebb0-aba7-4509-96f4-eed17fd31371","detail":{"errorcode":"policies.ratelimit.QuotaViolation"}}}


Fetching Articles:   2%|▏         | 7/366 [00:44<38:07,  6.37s/it]

Error: 429 - {"fault":{"faultstring":"Rate limit quota violation. Quota limit  exceeded. Identifier : 2f21ebb0-aba7-4509-96f4-eed17fd31371","detail":{"errorcode":"policies.ratelimit.QuotaViolation"}}}


Fetching Articles:   2%|▏         | 9/366 [01:02<41:14,  6.93s/it]


KeyboardInterrupt: 

In [ ]:
articledf.to_csv("/data/nyt_articles_2017.csv", index=False)